In [1]:
from typing import Any

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from mcp.types import (
    CallToolResult,
    EmbeddedResource,
    ImageContent,
    TextContent,
)
from mcp.types import (
    Tool as MCPTool,
)

from autogen.agentchat import AssistantAgent
from autogen.tools import Tool

In [2]:
# A bit modified langchain_mcp_adapters
# https://github.com/langchain-ai/langchain-mcp-adapters/blob/main/langchain_mcp_adapters/tools.py
NonTextContent = ImageContent | EmbeddedResource


def _convert_call_tool_result(
    call_tool_result: CallToolResult,
) -> tuple[str | list[str], list[NonTextContent] | None]:
    text_contents: list[TextContent] = []
    non_text_contents = []
    for content in call_tool_result.content:
        if isinstance(content, TextContent):
            text_contents.append(content)
        else:
            non_text_contents.append(content)

    tool_content: str | list[str] = [content.text for content in text_contents]
    if len(text_contents) == 1:
        tool_content = tool_content[0]

    if call_tool_result.isError:
        # raise ToolException(tool_content)
        raise ValueError(f"Tool call failed: {tool_content}")

    return tool_content, non_text_contents or None


def convert_mcp_tool_to_ag2_tool(
    session: ClientSession,
    tool: MCPTool,
) -> Tool:
    """Convert an MCP tool to a LangChain tool.

    NOTE: this tool can be executed only in a context of an active MCP client session.

    Args:
        session: MCP client session
        tool: MCP tool to convert

    Returns:
        a LangChain tool
    """

    async def call_tool(
        **arguments: dict[str, Any],
    ) -> tuple[str | list[str], list[NonTextContent] | None]:
        print(f"Arguments: {arguments}")

        call_tool_result = await session.call_tool(tool.name, arguments)
        return _convert_call_tool_result(call_tool_result)

    # return StructuredTool(
    #     name=tool.name,
    #     description=tool.description or "",
    #     args_schema=tool.inputSchema,
    #     coroutine=call_tool,
    #     response_format="content_and_artifact",
    # )
    return Tool(
        name=tool.name,
        description=tool.description or "",
        func_or_tool=call_tool,
    )


async def load_mcp_tools(session: ClientSession) -> list[Tool]:
    """Load all available MCP tools and convert them to LangChain tools."""
    tools = await session.list_tools()
    return [convert_mcp_tool_to_ag2_tool(session, tool) for tool in tools.tools]

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=["math_server.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)
        print(f"Len tools: {len(tools)}")
        print(f"Langchain tools 0: {tools[0].name}")
        print(f"Langchain tools 1: {tools[1].name}")

        f_result = await tools[0].func(a=2, b=5)
        print(f"Result from Langchain tool 0: {f_result}")

        agent = AssistantAgent(name="assistant", llm_config={"model": "gpt-4o-mini", "api_type": "openai"})
        for tool in tools:
            tool.register_for_llm(agent)

        print(agent.llm_config["tools"])
        # agent.run(
        #     message="Add 123223 and 456789",
        #     tools=tools,
        #     max_turns=2,
        #     user_input=False,
        # )

Len tools: 2
Langchain tools 0: add
Langchain tools 1: multiply
Arguments: {'a': 2, 'b': 5}
Result from Langchain tool 0: ('7', None)
[{'type': 'function', 'function': {'description': 'Add two numbers', 'name': 'add', 'parameters': {'type': 'object', 'properties': {'arguments': {'type': 'object', 'description': 'arguments'}}, 'required': ['arguments']}}}, {'type': 'function', 'function': {'description': 'Multiply two numbers', 'name': 'multiply', 'parameters': {'type': 'object', 'properties': {'arguments': {'type': 'object', 'description': 'arguments'}}, 'required': ['arguments']}}}]
